# MXNet Model Server

This Notebook is borrowed from: https://github.com/TalkAI/facial-emotion-recognition-gluon/blob/master/notebooks/Gluon_FERPlus.ipynb

## Install the required Packages locally

We will need the PyPi packages listed below to test model server locally, and to perform image pre-processing prior to the model inference.

In [1]:
!pip install mxnet-model-server
!pip install scikit-image
!pip install opencv-python

  Using cached https://files.pythonhosted.org/packages/4a/ae/6bda14f740dbfa608cfdbb4d0d9337bb8681fb1692f8fd2b418df07f3fd3/mxnet_model_server-0.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/16/7d/a8bb127562b12c5ac285f937af8662db03b59089db2bded68461790e1880/mxnet_mkl-1.3.0.post0-py2.py3-none-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/41/ff/9dedf5ec330be64c6c5f7b13e31d7ff674eb3525a2cc73c9f605fedb2e84/onnx-1.1.1.tar.gz
  Running setup.py bdist_wheel for onnx ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/fb/13/8e/b5652eb574597bfed05b000ed359413029ef2391f24315b733
Successfully built onnx
distributed 1.21.8 requires msgpack, which is not installed.
  Found existing installation: onnx 1.3.0
    Uninstalling onnx-1.3.0:
      Successfully uninstalled onnx-1.3.0
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
  Using 

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
  Using cached https://files.pythonhosted.org/packages/18/7f/c836c44ab30074a8486e30f8ea6adc8e6ac02332851ab6cc069e2ac35b84/opencv_python-3.4.3.18-cp36-cp36m-manylinux1_x86_64.whl
distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
# We define the model's input and output type and shape via signature.json
!cat ./model_archive_resources/signature.json

{
  "input_type": "image/jpeg", 
  "inputs": [
    {
      "data_shape": [1, 1, 64, 64], 
      "data_name": "data"
    }
  ], 
  "outputs": [
    {
      "data_shape": [1, 8],
      "data_name": "hybridsequential0_dense2_fwd"
    }
  ], 
  "output_type": "application/json"
}

In [2]:
# We define the model's class label names via synset.txt
!cat ./model_archive_resources/synset.txt

neutral
happiness
surprise
sadness
anger
disgust
fear
contempt

In [3]:
# And lastly, we define custom code for request handling via python code other auxiliary files
!cat ./model_archive_resources/fer_service.py

import numpy as np
from mms.utils.mxnet import image
from mms.model_service.mxnet_model_service import MXNetBaseService
from skimage import transform
import mxnet as mx
import cv2 as cv

# One time initialization of Haar Cascade Classifier to extract and crop out face
face_detector = cv.CascadeClassifier('haarcascade_frontalface.xml')
# Classifier parameter specifying how much the image size is reduced at each image scale
scale_factor = 1.3
# Classifier parameter how many neighbors each candidate rectangle should have to retain it
min_neighbors = 5

def crop_face(image):
    """Attempts to identify a face in the input image.

    Parameters
    ----------
    image : array representing a BGR image

    Returns
    -------
    array
        The cropped face, transformed to grayscale. If no face found returns None

    """
    gray_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    face_roi_list = face_detector.detectMultiScale(gray_image, scale_factor, min_neighbors)
    
    if (len(fac

## Download Model

In [4]:
%cd model_archive_resources
!curl -O https://s3.amazonaws.com/mxnet-demo-models/models/fer/gluon_ferplus-0000.params
!curl -O https://s3.amazonaws.com/mxnet-demo-models/models/fer/gluon_ferplus-symbol.json

/Users/wamy/nswamy/deepengine/workspace/dl-introduction-apache-mxnet-odsc-2018/part3_model_server/model_archive_resources
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 33.4M  100 33.4M    0     0  7257k      0  0:00:04  0:00:04 --:--:-- 7429k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 16867  100 16867    0     0      0      0 --:--:-- --:--:-- --:--:--     0 0  36351      0 --:--:-- --:--:-- --:--:-- 36273


## Export model

In [ ]:
%cd ../
!pwd

In [12]:
# Let's package everything up into a Model Archive bundle
!mxnet-model-export --model-name ferplus \
--service-file-path ./model_archive_resources/fer_service.py \
--model-path ./model_archive_resources

In [13]:
!ls -l ferplus.model

-rw-r--r--  1 wamy  1896053708  71970001 Oct 30 00:05 ferplus.model


In [16]:
# Spawning a new process to run the server
import subprocess as sp
server = sp.Popen("mxnet-model-server --models ferplus=ferplus.model", shell=True)

In [18]:
# Check out the health endpoint
!curl http://localhost:8080/ping

{"health":"healthy!"}


In [20]:
!curl -X POST http://localhost:8080/ferplus/predict -F "data=@./happy.jpg"

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>Error occurs while inference was executed on server.</p>


In [19]:
# Lastly, we'll terminate the server
server.terminate()